## Final

In [1]:
# Dependencies
import json
import requests as req
import pandas as pd
import os
import plotly as py
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from census import Census
from states import states, states_hash
from passwords import census_key, google_key, plotly_user, plotly_key
from NCMEC_parser import NCMEC_parse

ModuleNotFoundError: No module named 'plotly'

### Pulling Data from NCMEC

In [ ]:
# NCMEC website parser by state - NCMEC_parser.py
for state in states:
    NCMEC_parser(state)

### Combining and Cleaning NCMEC Data

In [2]:
# combining data by state in to one df
df = pd.DataFrame(columns = ['name','age','gender','ethnicity','NCMEC_case#','case_type','birth_year','missing_year','missing_city','missing_county','missing_state'])

index = 0
for state in states:
    state_df = pd.read_csv('Resources/NCMEC_{}.csv'.format(state))
    for i in range(len(state_df)):
        df.set_value(index,'name',state_df['name'][i])
        df.set_value(index,'age',state_df['age'][i])
        df.set_value(index,'gender',state_df['gender'][i])
        df.set_value(index,'ethnicity',state_df['ethnicity'][i])
        df.set_value(index,'NCMEC_case#',state_df['NCMEC_case#'][i])
        df.set_value(index,'case_type',state_df['case_type'][i])
        df.set_value(index,'birth_year',state_df['birth_year'][i])
        df.set_value(index,'missing_year',state_df['missing_year'][i])
        df.set_value(index,'missing_city',state_df['missing_city'][i])
        df.set_value(index,'missing_county',state_df['missing_county'][i])
        df.set_value(index,'missing_state',states[state])
        index += 1
df

,name,age,gender,ethnicity,NCMEC_case#,case_type,birth_year,missing_year,missing_city,missing_county,missing_state
0,DAVID BORER,36,male,white,722859,nonFamilyAbduct,1981,1989,WILLOW,Matanuska Susitna,Alaska
1,JANE DOE1980,-1,female,white,1184520,unidentified,NaN,NaN,NaN,NaN,Alaska
2,MEGAN EMERICK,61,female,white,1080580,endangeredMissing,1956,1973,SEWARD,Kenai Peninsula,Alaska
3,STELLA EVON,38,female,amInd,919705,endangeredMissing,1979,1996,BETHEL,Bethel,Alaska
4,AMY FANDEL,47,female,white,601234,nonFamilyAbduct,1970,1978,STERLING,Kenai Peninsula,Alaska
5,AMY FANDEL,47,female,white,601234,nonFamilyAbduct,1970,1978,STERLING,Kenai Peninsula,Alaska
6,ISAIAH ISENBERG,16,male,amInd,1306916,endangeredRunaway,2001,2017,ANCHORAGE,Anchorage,Alaska
7,DASHAWN MCCORMICK,10,male,biracial,1228477,endangeredMissing,2007,2013,ANCHORAGE,Anchorage,Alaska
8,MICHAEL PALMER,33,male,white,864936,endangeredMissing,1984,1999,WASILLA,Matanuska-Susitna Borough,Alaska
9,ISAIAH PITKA,14,male,amInd,1308930,endangeredRunaway,2003,2017,NORTH POLE,Fairbanks North Star,Alaska


In [34]:
# cleaning combined NCMEC data for 50 states
df['age'] = df['age'].replace(-1,float('NaN'))
df['gender'] = df['gender'].replace('unknown',float('NaN'))
df['ethnicity'] = df['ethnicity'].replace('unknown',float('NaN'))
df['ethnicity'] = df['ethnicity'].replace('amInd','native american')
df['ethnicity'] = df['ethnicity'].replace(['blackHisp','whiteHisp'],'hispanic')
df['case_type'] = df['case_type'].replace(['unidentified','unknown'],float('NaN'))
df['case_type'] = df['case_type'].replace('endangeredRunaway','runaway')
df['case_type'] = df['case_type'].replace(['Section 5779','endangeredMissing','lostInjuredMissing'],'missing')
df['case_type'] = df['case_type'].replace(['familyAbduct','hagueCase','nonFamilyAbduct'],'abduct')
df.head()

,name,age,gender,ethnicity,NCMEC_case#,case_type,birth_year,missing_year,missing_city,missing_county,missing_state
0,DAVID BORER,36.0,male,white,722859,abduct,1981,1989,WILLOW,Matanuska Susitna,Alaska
1,JANE DOE1980,NaN,female,white,1184520,NaN,NaN,NaN,NaN,NaN,Alaska
2,MEGAN EMERICK,61.0,female,white,1080580,missing,1956,1973,SEWARD,Kenai Peninsula,Alaska
3,STELLA EVON,38.0,female,native american,919705,missing,1979,1996,BETHEL,Bethel,Alaska
4,AMY FANDEL,47.0,female,white,601234,abduct,1970,1978,STERLING,Kenai Peninsula,Alaska


In [39]:
# finding latitude and longitude for missing city,state
# for i in range(len(df)):
#     try:
#         gkey = google_key
#         target_city = "{}, {}".format(df['missing_city'][i],df['missing_state'][i])
#         target_url = "https://maps.googleapis.com/maps/api/geocode/json?address={}&key={}".format(target_city, gkey)
#         geo_data = req.get(target_url).json()
#         lat = geo_data["results"][0]["geometry"]["location"]["lat"]
#         lng = geo_data["results"][0]["geometry"]["location"]["lng"]
#         print("location: {} lat: {} lng: {}".format(target_city,lat,lng))
#     except:
#         print("{} fail".format(i))
#         break
         


lat: 61.7697336 lng: -149.9910633
lat: 70.41049579999999 lng: -148.9721164
lat: 60.10416669999999 lng: -149.4422222
lat: 60.7922222 lng: -161.7558334
lat: 60.53722219999999 lng: -150.7647222
lat: 60.53722219999999 lng: -150.7647222
lat: 61.2180556 lng: -149.9002778
lat: 61.2180556 lng: -149.9002778
lat: 61.5813889 lng: -149.4394444
lat: 64.7511111 lng: -147.3494443
lat: 61.5997222 lng: -149.1127777
lat: 61.2180556 lng: -149.9002778
lat: 57.79000000000001 lng: -152.4072221
lat: 32.3668052 lng: -86.2999689
lat: 30.6953657 lng: -88.0398912
lat: 30.381901 lng: -87.4356227
lat: 32.4073589 lng: -87.02110069999999
lat: 32.6454116 lng: -85.3782795
lat: 32.8895681 lng: -86.2196936
lat: 32.430237 lng: -85.7077266
lat: 33.4512197 lng: -86.9966606
21 fail
lat: 30.6953657 lng: -88.0398912
lat: 34.5789747 lng: -85.986644
lat: 30.5478002 lng: -88.1903799
lat: 30.5478002 lng: -88.1903799
lat: 30.5478002 lng: -88.1903799
lat: 33.2556719 lng: -87.3208336
lat: 34.7303688 lng: -86.5861037
lat: 33.6444926 

lat: 33.5386523 lng: -112.1859866
lat: 37.7799273 lng: -121.9780153
lat: 33.8365932 lng: -117.9143012
lat: 34.0522342 lng: -118.2436849
lat: 34.1898566 lng: -118.451357
lat: 34.6867846 lng: -118.1541632
lat: 34.1898566 lng: -118.451357
lat: 34.1898566 lng: -118.451357
lat: 33.7857948 lng: -118.2643567
lat: 33.7857948 lng: -118.2643567
lat: 34.0522342 lng: -118.2436849
lat: 34.0686206 lng: -118.0275667
lat: 34.1898566 lng: -118.451357
lat: 37.7799273 lng: -121.9780153
lat: 34.1898566 lng: -118.451357
lat: 34.235822 lng: -118.4849365
lat: 34.2767691 lng: -118.4104684
lat: 34.055103 lng: -117.7499909
lat: 33.9816812 lng: -118.2250725
lat: 38.5815719 lng: -121.4943996
lat: 38.5815719 lng: -121.4943996
lat: 37.3382082 lng: -121.8863286
lat: 37.3382082 lng: -121.8863286
lat: 34.1477849 lng: -118.1445155
lat: 37.8651011 lng: -119.5383294
lat: 34.0805651 lng: -118.072846
lat: 33.7700504 lng: -118.1937395
lat: 34.0633443 lng: -117.6508876
lat: 34.1008426 lng: -117.7678355
lat: 34.0686206 lng: -

lat: 34.0522342 lng: -118.2436849
lat: 40.448208 lng: -122.2977815
lat: 34.055103 lng: -117.7499909
lat: 34.055103 lng: -117.7499909
lat: 35.3732921 lng: -119.0187125
lat: 34.1898566 lng: -118.451357
lat: 33.1433723 lng: -117.1661449
lat: 34.2504595 lng: -118.4338345
lat: 33.9533487 lng: -117.3961564
lat: 34.2793576 lng: -118.5021527
lat: 33.9533487 lng: -117.3961564
lat: 37.63909719999999 lng: -120.9968782
lat: 34.0224417 lng: -118.1669736
lat: 38.8965654 lng: -121.0768901
lat: 33.9424658 lng: -117.2296717
lat: 38.440429 lng: -122.7140548
lat: 34.0522342 lng: -118.2436849
lat: 38.24935809999999 lng: -122.0399663
lat: 34.1898566 lng: -118.451357
lat: 33.8752935 lng: -117.5664384
lat: 33.8752935 lng: -117.5664384
lat: 38.5890723 lng: -121.302728
lat: 34.1066756 lng: -117.8067257
lat: 35.1321877 lng: -118.4489739
lat: 34.68824560000001 lng: -120.4364208
lat: 34.0522342 lng: -118.2436849
lat: 34.0522342 lng: -118.2436849
lat: 35.3732921 lng: -119.0187125
lat: 36.3302284 lng: -119.2920585


lat: 33.6469437 lng: -117.6861023
lat: 37.5629917 lng: -122.3255254
lat: 34.0211224 lng: -118.3964665
lat: 34.1008426 lng: -117.7678355
lat: 37.7749295 lng: -122.4194155
lat: 33.7454725 lng: -117.867653
lat: 33.7454725 lng: -117.867653
lat: 33.7454725 lng: -117.867653
lat: 34.055103 lng: -117.7499909
lat: 34.235822 lng: -118.4849365
lat: 34.0211224 lng: -118.3964665
lat: 34.0211224 lng: -118.3964665
lat: 33.7454725 lng: -117.867653
lat: 37.63909719999999 lng: -120.9968782
lat: 33.9761238 lng: -117.9053395
lat: 34.0522342 lng: -118.2436849
lat: 34.208254 lng: -118.6058609
lat: 34.0522342 lng: -118.2436849
lat: 38.8965654 lng: -121.0768901
lat: 37.7749295 lng: -122.4194155
lat: 32.739491 lng: -114.6346751
lat: 33.2000368 lng: -117.2425355
lat: 34.0522342 lng: -118.2436849
lat: 38.8079573 lng: -122.5583155
lat: 34.0200114 lng: -117.9495083
lat: 33.7700504 lng: -118.1937395
lat: 34.0522342 lng: -118.2436849
lat: 37.1305012 lng: -121.6543901
lat: 33.930293 lng: -118.2114603
lat: 34.0522342 

lat: 41.8186802 lng: -72.47903699999999
lat: 41.4570108 lng: -72.8230736
lat: 41.8186802 lng: -72.47903699999999
lat: 39.158168 lng: -75.5243682
lat: 39.9525839 lng: -75.1652215
lat: 39.158168 lng: -75.5243682
lat: 39.6830978 lng: -75.7509725
lat: 39.0712238 lng: -75.47603269999999
lat: 39.6837226 lng: -75.74965720000002
lat: 39.0084463 lng: -75.5779807
lat: 39.5778901 lng: -75.588815
lat: 39.7390721 lng: -75.5397878
lat: 39.7390721 lng: -75.5397878
lat: 28.3644485 lng: -82.69343429999999
lat: 25.7616798 lng: -80.1917902
lat: 30.4382559 lng: -84.28073289999999
lat: 27.9462615 lng: -82.2420317
lat: 29.2858129 lng: -81.0558894
lat: 26.6253497 lng: -81.6248026
lat: 26.7153424 lng: -80.0533746
lat: 24.726026 lng: -81.0446262
lat: 25.7616798 lng: -80.1917902
lat: 27.4989278 lng: -82.5748194
lat: 27.4989278 lng: -82.5748194
lat: 26.640628 lng: -81.8723084
lat: 28.6988853 lng: -81.30812259999999
lat: 28.5383355 lng: -81.3792365
lat: 27.950575 lng: -82.4571776
lat: 26.007765 lng: -80.2962555
l

...

lat: 39.41426879999999 lng: -77.4105409
lat: 39.41426879999999 lng: -77.4105409
lat: 38.9559442 lng: -76.9455301
lat: 39.2903848 lng: -76.6121893
lat: 39.3794196 lng: -76.45990429999999
lat: 39.1434406 lng: -77.2013705
lat: 39.1434406 lng: -77.2013705
lat: 39.1434406 lng: -77.2013705
lat: 39.1434406 lng: -77.2013705
lat: 38.6265124 lng: -76.9105483
lat: 39.2903848 lng: -76.6121893
lat: 39.2903848 lng: -76.6121893
lat: 39.6528654 lng: -78.7625185
lat: 38.99066570000001 lng: -77.026088
lat: 38.99066570000001 lng: -77.026088
lat: 38.984652 lng: -77.0947092
lat: 39.1626084 lng: -76.6246886
lat: 39.3826031 lng: -77.26942780000002
lat: 39.3826031 lng: -77.26942780000002
lat: 39.1073317 lng: -76.57107529999999
lat: 39.2903848 lng: -76.6121893
lat: 39.2903848 lng: -76.6121893
lat: 39.2903848 lng: -76.6121893
lat: 39.60677889999999 lng: -75.83327179999999
lat: 39.5359406 lng: -76.34829309999999
lat: 39.1378733 lng: -76.8217374
lat: 39.1378733 lng: -76.8217374
lat: 39.0398314 lng: -77.0552555000

lat: 39.0997265 lng: -94.5785667
lat: 37.49504780000001 lng: -94.27661479999999
lat: 39.7674578 lng: -94.84668099999999
lat: 37.2089572 lng: -93.29229889999999
lat: 37.176447 lng: -94.31022279999999
lat: 38.8106075 lng: -90.69984769999999
lat: 36.970891 lng: -93.717979
lat: 38.8858407 lng: -94.5330096
lat: 40.2888803 lng: -95.0785525
lat: 37.08422710000001 lng: -94.51328099999999
lat: 39.04643859999999 lng: -90.742347
lat: 38.2081045 lng: -91.16042069999999
lat: 36.7569994 lng: -90.392888
lat: 39.0997265 lng: -94.5785667
lat: 38.34836360000001 lng: -92.58157589999999
lat: 37.7242202 lng: -89.8612196
lat: 37.7242202 lng: -89.8612196
lat: 37.1036665 lng: -93.81854009999999
lat: 39.0911161 lng: -94.41550679999999
lat: 39.0997265 lng: -94.5785667
lat: 37.2089572 lng: -93.29229889999999
lat: 39.0997265 lng: -94.5785667
lat: 38.9517053 lng: -92.3340724
lat: 38.6270025 lng: -90.19940419999999
lat: 38.789217 lng: -90.322614
lat: 39.6536529 lng: -91.7346056
lat: 36.8689554 lng: -94.367995999999

lat: 39.3642834 lng: -74.4229266
lat: 40.2005472 lng: -74.03141649999999
lat: 39.3642834 lng: -74.4229266
lat: 40.767323 lng: -74.2048677
lat: 40.8042674 lng: -74.012084
lat: 39.532777 lng: -75.3761952
lat: 40.6337136 lng: -74.4073736
lat: 39.9583094 lng: -74.6282653
lat: 40.735657 lng: -74.1723667
lat: 40.8214894 lng: -73.9876388
lat: 34.4047987 lng: -103.2052272
lat: 35.2327544 lng: -106.6630437
lat: 35.0853336 lng: -106.6055534
lat: 35.5280783 lng: -108.7425843
lat: 35.0853336 lng: -106.6055534
lat: 35.0853336 lng: -106.6055534
lat: 35.0853336 lng: -106.6055534
lat: 32.7609645 lng: -107.905471
lat: 32.7609645 lng: -107.905471
lat: 32.7609645 lng: -107.905471
lat: 32.3199396 lng: -106.7636538
lat: 35.0853336 lng: -106.6055534
lat: 32.7026116 lng: -103.1360403
lat: 32.3199396 lng: -106.7636538
lat: 35.0853336 lng: -106.6055534
lat: 32.0513075 lng: -106.4288088
lat: 35.6869752 lng: -105.937799
lat: 35.0853336 lng: -106.6055534
lat: 36.1988885 lng: -105.888532
lat: 35.0853336 lng: -106.

2917 fail
2918 fail
2919 fail
2920 fail
2921 fail
2922 fail
2923 fail
2924 fail
2925 fail
2926 fail
2927 fail
2928 fail
2929 fail
2930 fail
2931 fail
2932 fail
2933 fail
2934 fail
2935 fail
2936 fail
2937 fail
2938 fail
2939 fail
2940 fail
2941 fail
2942 fail
2943 fail
2944 fail
2945 fail
2946 fail
2947 fail
2948 fail
2949 fail
2950 fail
2951 fail
2952 fail
2953 fail
2954 fail
2955 fail
2956 fail
2957 fail
2958 fail
2959 fail
2960 fail
2961 fail
2962 fail
2963 fail
2964 fail
2965 fail
2966 fail
2967 fail
2968 fail
2969 fail
2970 fail
2971 fail
2972 fail
2973 fail
2974 fail
2975 fail
2976 fail
2977 fail
2978 fail
2979 fail
2980 fail
2981 fail
2982 fail
2983 fail
2984 fail
2985 fail
2986 fail
2987 fail
2988 fail
2989 fail
2990 fail
2991 fail
2992 fail
2993 fail
2994 fail
2995 fail
2996 fail
2997 fail
2998 fail
2999 fail
3000 fail
3001 fail
3002 fail
3003 fail
3004 fail
3005 fail
3006 fail
3007 fail
3008 fail
3009 fail
3010 fail
3011 fail
3012 fail
3013 fail
3014 fail
3015 fail
3016 fail


3737 fail
3738 fail
3739 fail
3740 fail
3741 fail
3742 fail
3743 fail
3744 fail
3745 fail
3746 fail
3747 fail
3748 fail
3749 fail
3750 fail
3751 fail
3752 fail
3753 fail
3754 fail
3755 fail
3756 fail
3757 fail
3758 fail
3759 fail
3760 fail
3761 fail
3762 fail
3763 fail
3764 fail
3765 fail
3766 fail
3767 fail
3768 fail
3769 fail
3770 fail
3771 fail
3772 fail
3773 fail
3774 fail
3775 fail
3776 fail
3777 fail
3778 fail
3779 fail
3780 fail
3781 fail
3782 fail
3783 fail
3784 fail
3785 fail
3786 fail
3787 fail
3788 fail
3789 fail
3790 fail
3791 fail
3792 fail
3793 fail
3794 fail
3795 fail
3796 fail
3797 fail
3798 fail
3799 fail
3800 fail
3801 fail
3802 fail
3803 fail
3804 fail
3805 fail
3806 fail
3807 fail
3808 fail
3809 fail
3810 fail
3811 fail
3812 fail
3813 fail
3814 fail
3815 fail
3816 fail
3817 fail
3818 fail
3819 fail
3820 fail
3821 fail
3822 fail
3823 fail
3824 fail
3825 fail
3826 fail
3827 fail
3828 fail
3829 fail
3830 fail
3831 fail
3832 fail
3833 fail
3834 fail
3835 fail
3836 fail


In [ ]:
df.to_csv("Resources/_NCMEC_final.csv", encoding="utf-8", index=False)

### Missing Peoples Ratio - # of Missing People vs State Population

In [4]:
# Census API Key
c = Census(census_key)

# Read Main CSV
msgPpl = pd.read_csv("Resources/_NCMEC_final.csv")
msgPpl.head()

,name,age,gender,ethnicity,NCMEC_case#,case_type,birth_year,missing_year,missing_city,missing_county,missing_state
0,DAVID BORER,36.0,male,white,722859,abduct,1981.0,1989.0,WILLOW,Matanuska Susitna,Alaska
1,JANE DOE1980,NaN,female,white,1184520,NaN,NaN,NaN,NaN,NaN,Alaska
2,MEGAN EMERICK,61.0,female,white,1080580,missing,1956.0,1973.0,SEWARD,Kenai Peninsula,Alaska
3,STELLA EVON,38.0,female,native american,919705,missing,1979.0,1996.0,BETHEL,Bethel,Alaska
4,AMY FANDEL,47.0,female,white,601234,abduct,1970.0,1978.0,STERLING,Kenai Peninsula,Alaska


In [5]:
# Pull State level census data
census_data1 = c.acs5.get(("NAME", "B19013_001E", "B01003_001E", "B01002_001E",
                          "B19301_001E",
                          "B17001_002E"), {'for': 'zip code tabulation area:*'})

url1 = "https://api.census.gov/data/2014/pep/natstprc?get=STNAME,POP&for=state:*&DATE=7"
reqUrl1 = url1 + "&key="+ census_key
getData1 = req.get(reqUrl1).json()

In [9]:
# State level dataframe
df_states = pd.DataFrame(getData1,columns=getData1[0])
df_states = df_states.drop(df_states.index[0])
df_states['POP'] =  list(map(int,df_states['POP']))
df_states = df_states.drop('DATE', 1)
df_states = df_states[df_states.STNAME != 'District of Columbia']
df_states = df_states[df_states.STNAME != 'Puerto Rico Commonwealth']
df_states = df_states.reset_index(drop = True)
df_states['state_key'] = [states_hash[state] for state in list(df_states['STNAME'])]
df_states['Miss_Pop'] = list(msgPpl.groupby('missing_state').count()['name'])
# Devided population by 1 million for a better ratio between missing peoples and population
df_states['Miss%'] = round(df_states['Miss_Pop']/df_states['POP'] * 1000000,2)

df_states

,STNAME,POP,state,state_key,Miss_Pop,Miss%
0,Alabama,4849377,01,AL,36,7.42
1,Alaska,736732,02,AK,13,17.65
2,Arizona,6731484,04,AZ,153,22.73
3,Arkansas,2966369,05,AR,39,13.15
4,California,38802500,06,CA,623,16.06
5,Colorado,5355866,08,CO,85,15.87
6,Connecticut,3596677,09,CT,25,6.95
7,Delaware,935614,10,DE,10,10.69
8,Florida,19893297,12,FL,360,18.10
9,Georgia,10097343,13,GA,99,9.80


In [10]:
url2 = "https://api.census.gov/data/2015/acs1?get=NAME,B19013_001E,B02001_002E,B02001_003E,B02001_004E,B02001_005E,B02001_006E,B02001_007E,B02001_008E,B02001_009E,B02001_010E&for=state:*"
reqUrl2 = url2 + "&key="+ census_key
getData2 = req.get(reqUrl2).json()

In [11]:
df_temp = pd.DataFrame(getData2, columns=getData2[0])
df_temp = df_temp.drop(df_temp.index[0])
df_temp = df_temp.drop('state', 1)
# getIncData["Total Ethnic"] =  int (getIncData["B02001_002E"]) + int (getIncData["B02001_003E"]) +int (getIncData["B02001_004E"]) + int (getIncData["B02001_005E"]) + int (getIncData["B02001_006E"]) + int (getIncData["B02001_007E"]) + int (getIncData["B02001_008E"]) + int (getIncData["B02001_009E"]) + int (getIncData["B02001_010E"])
df_temp = df_temp.rename(columns={"B19013_001E": "Household Income", 
                                      "B02001_002E": "Whites", 
                                      "B02001_003E": "African Americans",
                                      "B02001_004E": "American Indians",
                                      "B02001_005E": "Asians",
                                      "B02001_006E": "Pacific Islanders",
                                      "B02001_007E": "Others",
                                      "B02001_008E": "Two or More Races",
                                      "B02001_009E": "Two or More Inc Others",
                                      "B02001_010E": "Two or More Excl Others",
                                      "NAME": "STNAME" })

df_temp = df_temp[df_temp.STNAME != 'District of Columbia']
df_temp = df_temp[df_temp.STNAME != 'Puerto Rico Commonwealth']
df_temp = df_temp[df_temp.STNAME != 'Puerto Rico']
df_temp = df_temp.reset_index(drop = True)
df_temp

,STNAME,Household Income,Whites,African Americans,American Indians,Asians,Pacific Islanders,Others,Two or More Races,Two or More Inc Others,Two or More Excl Others
0,Alabama,44765,3329913,1303298,20503,59393,4448,54105,87319,6354,80965
1,Alaska,73355,481314,26176,101133,47295,6762,12467,63285,3171,60114
2,Arizona,51492,5285994,300685,308469,217556,11284,481631,222446,45251,177195
3,Arkansas,41995,2308047,469555,20797,42404,7639,65542,64220,6149,58071
4,California,64500,23824254,2277229,282777,5548936,157554,5300297,1753771,375682,1378089
5,Colorado,63909,4607390,224809,49138,162880,9136,209890,193331,30981,162350
6,Connecticut,71346,2747943,380571,8893,156249,519,180107,116604,28025,88579
7,Delaware,61255,654904,203863,3121,36114,334,19157,28441,3500,24941
8,Florida,49426,15357374,3280778,47032,544068,11898,528266,501856,87649,414207
9,Georgia,51244,6077438,3197929,28491,388457,4337,284084,234124,36148,197976


In [12]:
df_states = df_states.merge(df_temp, on='STNAME')

In [13]:
# Clean data and add ethnic percentages
df_states['Whites'] = list(map(int,df_states['Whites']))
df_states['Whites%'] = round(df_states['Whites']/df_states['POP']*100, 2)
df_states['African Americans'] = list(map(int,df_states['African Americans']))
df_states['African Americans%'] = round(df_states['African Americans']/df_states['POP']*100, 2)
df_states['American Indians'] = list(map(int,df_states['American Indians']))
df_states['American Indians%'] = round(df_states['American Indians']/df_states['POP']*100, 2)
df_states['Asians'] = list(map(int,df_states['Asians']))
df_states['Asians%'] = round(df_states['Asians']/df_states['POP']*100, 2)
df_states['Pacific Islanders'] = list(map(int,df_states['Pacific Islanders']))
df_states['Pacific Islanders%'] = round(df_states['Pacific Islanders']/df_states['POP']*100, 2)
df_states['Others'] = list(map(int,df_states['Others']))
df_states['Others%'] = round(df_states['Others']/df_states['POP']*100, 2)
df_states.rename(columns={'Two or More Races': 'BiRacial','Two or More Inc Others': 'BiRacial Incl Others','Two or More Excl Others':'BiRacial Excl Others'}, inplace=True)
df_states['BiRacial'] = list(map(int,df_states['BiRacial']))
df_states['BiRacial%'] = round(df_states['BiRacial']/df_states['POP']*100, 2)
df_states['BiRacial Incl Others'] = list(map(int,df_states['BiRacial Incl Others']))
df_states['BiRacial Excl Others'] = list(map(int,df_states['BiRacial Excl Others']))
df_states


'''
Gender + Education level Data if possible
'''

,STNAME,POP,state,state_key,Miss_Pop,Miss%,Household Income,Whites,African Americans,American Indians,...,BiRacial,BiRacial Incl Others,BiRacial Excl Others,Whites%,African Americans%,American Indians%,Asians%,Pacific Islanders%,Others%,BiRacial%
0,Alabama,4849377,01,AL,36,7.42,44765,3329913,1303298,20503,...,87319,6354,80965,68.67,26.88,0.42,1.22,0.09,1.12,1.80
1,Alaska,736732,02,AK,13,17.65,73355,481314,26176,101133,...,63285,3171,60114,65.33,3.55,13.73,6.42,0.92,1.69,8.59
2,Arizona,6731484,04,AZ,153,22.73,51492,5285994,300685,308469,...,222446,45251,177195,78.53,4.47,4.58,3.23,0.17,7.15,3.30
3,Arkansas,2966369,05,AR,39,13.15,41995,2308047,469555,20797,...,64220,6149,58071,77.81,15.83,0.70,1.43,0.26,2.21,2.16
4,California,38802500,06,CA,623,16.06,64500,23824254,2277229,282777,...,1753771,375682,1378089,61.40,5.87,0.73,14.30,0.41,13.66,4.52
5,Colorado,5355866,08,CO,85,15.87,63909,4607390,224809,49138,...,193331,30981,162350,86.03,4.20,0.92,3.04,0.17,3.92,3.61
6,Connecticut,3596677,09,CT,25,6.95,71346,2747943,380571,8893,...,116604,28025,88579,76.40,10.58,0.25,4.34,0.01,5.01,3.24
7,Delaware,935614,10,DE,10,10.69,61255,654904,203863,3121,...,28441,3500,24941,70.00,21.79,0.33,3.86,0.04,2.05,3.04
8,Florida,19893297,12,FL,360,18.10,49426,15357374,3280778,47032,...,501856,87649,414207,77.20,16.49,0.24,2.73,0.06,2.66,2.52
9,Georgia,10097343,13,GA,99,9.80,51244,6077438,3197929,28491,...,234124,36148,197976,60.19,31.67,0.28,3.85,0.04,2.81,2.32


In [14]:
# Plotly Info
py.tools.set_credentials_file(plotly_user, plotly_key)
init_notebook_mode(connected=True)

In [31]:
# Changing everything to strings
df_graph1 = df_states
for col in df_graph1.columns:
    df_graph1[col] = df_graph1[col].astype(str)

# Color Scale
scl = [[0.0, 'rgb(0,255,0)'],[0.2, 'rgb(100,255,0)'],[0.4, 'rgb(222,255,0)'],\
            [0.6, 'rgb(255,222,0)'],[0.8, 'rgb(255,100,0)'],[1.0, 'rgb(255,0,0)']]

# text overlay for states
df_graph1['text'] = "---" + df_graph1['STNAME'] + "---" + '<br>'\
            "Missing: " + df_graph1['Miss_Pop'] + '<br>'\
            "Population: " + df_graph1['POP'] + '<br>'\
            "Average Income: $" + df_graph1['Household Income'] + '<br>'\
            "---" + "Ethnic%" + "---" + '<br>'\
            "White: "+df_graph1['Whites%']+"% "+"Black: "+df_graph1['African Americans%']+"% "+":Native American: "+df_graph1['American Indians%']+"% "+"Asian: "+df_graph1['Asians%'] +"%" + '<br>'\
            "Pacific Islander: "+df_graph1['Pacific Islanders%']+"% "+"BiRacial: "+df_graph1['BiRacial%']+"% "+"Other: "+df_graph1['Others%']+"%"
            
'''
????
According to 2015 US Census Bureau estimates, California's population was 72.9% White, 
6.5% Black or African American, 14.7% Asian, 1.7% Native Americans, 0.5% Pacific Islander and 3.8% from two or more races. 
By ethnicity, 38.8% of the total population is Hispanic-Latino (of any race) and 61.4% Non-Hispanic (of any race). 
Hispanics are the largest ethnic group in California.
 - Wikipedia - 
'''

# Bunch of fancy settings for the plot
data = [ dict(
        type='choropleth',
        colorscale = scl,
        autocolorscale = False,
        locations = df_graph1['state_key'],
        z = df_graph1['Miss%'].astype(float),
        locationmode = 'USA-states',
        text = df_graph1['text'],
        marker = dict(
            line = dict (
                color = 'rgb(255,255,255)',
                width = 2
            ) ),
        colorbar = dict(
            title = "Ratio Scale")
        ) ]

# Plot layout
layout = dict(
        title = '# of Missing People vs State Population<br>(Hover for breakdown)',
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showlakes = True,
            lakecolor = 'rgb(255, 255, 255)'),
             )

fig = dict( data=data, layout=layout )
iplot(fig, filename='Output/0_MissingPeopleRatio.html')

### Trend By Decade

In [ ]:
'''
<80s
90s
00s
10s>
plot on US MAP 
options:
1. dots by lat,lng https://plot.ly/python/scatter-plots-on-maps/
    a. slider https://plot.ly/python/sliders/
2. bubble grows bigger by decade https://plot.ly/python/bubble-maps/
'''

### Top 3 Population vs Bottom 3 Population

In [ ]:
df_Bot3Pop = df_state_pop.sort_values(["POP"], ascending = True).head(3)
df_Bot3Pop

In [ ]:
df_Top3Pop = df_state_pop.sort_values(["POP"], ascending=False).head(3)
df_Top3Pop

In [ ]:
'''
state level data for top 3 and bot 3 states
gender, ethnicity, income, education

by county
crime, income, education

plot on state map seperated by county
dots (missing people) based on town
https://plot.ly/python/county-level-choropleth/
'''